# Creation of the MINKE-SPOT labeled audio clips

This notebook uses the annotations dataset to generate the labelled audio clip required by minkeSPOT (well, MINKE-SPOT really) to train, test and evaluate the CNN (see https://github.com/ChristianBergler/ORCA-SPOT).

The audio clips must follow the following template/format: Filename Template:
call/noise-XXX_ID_YEAR_TAPENAME_STARTTIME_ENDTIME.wav

 - 1st-Element: call/noise-XXX = call-XXX or noise-XXX for pointing out wether it is a target signal or any kind of noise signal. XXX is a placeholder for any kind of string which could be added for more specific label information, e.g. call-N9, noise-boat

 - 2nd-Element: ID = unique ID (natural number) to identify the audio clip

 - 3rd-Element: YEAR = year of the tape when it has been recorded

 - 4th-Element: TAPENAME = name of the recorded tape (has to be unique in order to do a proper data split into train, devel, test set by putting one tape only in only one of the three sets

 - 5th-Element: STARTTIME = start time of the audio clip in milliseconds with respect to the original recording (natural number)

 - 6th-Element: ENDTIME = end time of the audio clip in milliseconds with respect to the original recording(natural number)

Examples of valid filenames:
 - call-Orca-A12_929_2019_Rec-031-2018-10-19-06-59-59-ASWMUX231648_2949326_2949919
 Label Name=call-Orca-A12, ID=929, Year=2019, Tapename=Rec-031-2018-10-19-06-59-59-ASWMUX231648, Starttime in ms=2949326, Starttime in ms=2949919

 - noise-humanVoice_2381_2010_101BC_149817_150055.wav
 Label Name=noise-humanVoice, ID=2381, Year=2010, Tapename=101BC, Starttime in ms=149817, Starttime in ms=150055



## Input parameters

In [1]:
params = dict()
params['dataset_file_path'] = r'C:\Users\xavier.mouy\Documents\GitHub\minke-whale-dataset\datasets\Annotations_dataset_NN-HK-MW-HB_20220316T142143.nc'
params['out_dir'] = r'C:\Users\xavier.mouy\Documents\GitHub\minke-spot\datasets'
params['tape_name_timeframe'] = '%Y%m%d%H'
params['class_call'] = ['MW']
params['class_noise'] = ['HB', 'HK', 'HKP', 'HKPT', 'NN', 'NNS']
params['sanpling_rate_hz'] = 2000
params['clips_buffer_s'] = 1

params['separate_label_folders'] = True
params['spectro_on'] = True
params['spetro_on_npy'] = False

params['spetro_nfft'] = 256
params['spetro_frame'] = 256
params['spetro_inc'] = 128


## Script

In [2]:
# Imports
from ecosound.core.annotation import Annotation
from ecosound.core.metadata import DeploymentInfo
from ecosound.core.audiotools import Sound
import pandas as pd
from datetime import datetime
import os
import librosa
import librosa.display
import soundfile
import csv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.use('Agg')

In [3]:
def calc_spectro(y, sr, n_fft,hop_length, win_length):
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
    S = librosa.amplitude_to_db(S,ref=np.max)
    fig, ax = plt.subplots()
    img = librosa.display.specshow(S, hop_length=hop_length,y_axis='linear', x_axis='time', sr=sr,ax=ax)
    #img = librosa.display.specshow(S, hop_length=hop_length,y_axis='linear', x_axis='time', sr=sr,ax=ax)
    ax.set_title('Power spectrogram')
    fig.colorbar(img, ax=ax, format="%+2.0f dB")
    return fig, ax, S

In [4]:
# Load dataset
dataset = Annotation()
dataset.from_netcdf(params['dataset_file_path'])
dataset.data['label'] = dataset.data['label_class'] + dataset.data['label_subclass']

In [5]:
# labels
class_ID = [1] * len(params['class_call']) + [0] * len(params['class_noise']) # 1 = call, 0 = noise
class_labels = params['class_call'] + params['class_noise']

In [6]:
# create output folders for this dataset
current_dir_name = datetime.strftime(datetime.now(),'%Y%m%dT%H%M%S') + '_' + '-'.join(params['class_call']) + '_' + '-'.join(params['class_noise'])
current_out_dir = os.path.join(params['out_dir'],current_dir_name)
os.mkdir(current_out_dir)
if params['separate_label_folders']:
    for label in class_labels:
        os.mkdir(os.path.join(current_out_dir,label))

In [7]:
# writes input parameters as csv file for book keeping purposes
log_file = open(os.path.join(current_out_dir,"parameters.csv"), "w")
writer = csv.writer(log_file) 
for key, value in params.items():
    writer.writerow([key, value])
log_file.close()

In [8]:
#loop through each class_labels / class_ID
annot_unique_id = 0
for sp_label, cl_id in zip(class_labels, class_ID):
    print(sp_label, cl_id)
    annot_sp = dataset.data[dataset.data['label'] == sp_label]
    # loop through is annot for that class label
    for idx, annot in annot_sp.iterrows():
        # define start/stop times +/- buffer
        t1 = annot.time_min_offset - params['clips_buffer_s']
        if t1 <= 0:
            t1=0
        t2 = annot.time_max_offset + params['clips_buffer_s']
        duration = t2-t1
        # load sound clip
        y, s = librosa.load(os.path.join(annot['audio_file_dir'],
                           annot['audio_file_name'])+annot['audio_file_extension'],
                           sr=params['sanpling_rate_hz'],
                           offset=t1,
                           duration=duration
                           ) # Downsample 44.1kHz to 8kHz
        t2 = t1+(len(y)/s) # readjust end time in case it exceeded the end  of the file (which librosa handles by taking the last sample)
       
        # Create audio clip standard name
        F=[None] * 6
        # Field 1: call/noise
        if cl_id == 1: # call
            F[0] = 'call-' + annot['label']
        elif cl_id == 0: # noise
            F[0] = 'noise-' + annot['label']
        # Field 2: unique ID
        F[1] = str(annot_unique_id)  
        # Field 3: YEAR
        F[2] = str(annot['time_min_date'].year)
        # Field 4: TAPENAME
        F[3] = ''.join(annot['deployment_ID']).replace("_", "-") + '-' + datetime.strftime(annot['time_min_date'],params['tape_name_timeframe'])
        # Field 5: STARTTIME in milliseconds
        F[4] = str(round(t1*1000))
        # Field 6: ENDTIME in milliseconds
        F[5] = str(round(annot.time_max_offset*1000))
        F = '_'.join(F) + '.wav'

        # Write audio clip
        outfilename = os.path.join(os.path.join(current_out_dir,sp_label,F))
        soundfile.write(outfilename, y, s, subtype=None, endian=None, format=None, closefd=True)
        
        # write spectrogram image
        if params['spectro_on']:
            fig, ax, S = calc_spectro(y,s, params['spetro_nfft'],params['spetro_inc'], win_length= params['spetro_frame'])
            fig.savefig(os.path.splitext(outfilename)[0] + '.png',dpi=600)
            plt.close('all') 
            plt.close()
            plt.cla()
            plt.clf()
            if params['spetro_on_npy']:
                np.save(os.path.splitext(outfilename)[0] + '.npy', S)
            
        annot_unique_id += 1

MW 1


C:\Users\xavier.mouy\Anaconda3\envs\ecosound\lib\site-packages\librosa\core\spectrum.py:1641: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn(


HB 0
HK 0
HKP 0
HKPT 0
NN 0


C:\Users\xavier.mouy\Anaconda3\envs\ecosound\lib\site-packages\librosa\core\spectrum.py:1641: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn(


NNS 0
